# Disclaimer

Source code has been adapted from https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5

# Source 

In [6]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
%matplotlib inline
import matplotlib
from IPython.display import display
import matplotlib.pyplot as plt

import numpy as np
#import cPickle as pickle
import _pickle as pickle
import gym
import logging, gzip, configparser

## Auxillary functions

In [7]:
def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def dsigmoid(x):
    """ this is the derivative of sigmoid """
    ex = exp(x)
    return ex / ((ex + 1.0)^2.0)

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I
  # return I.astype(np.float).ravel()


## Policy gradients

### Forward pass

In [8]:
def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h <= 0] = 0 # ReLU nonlinearity, no inhibitory neurons
    rho = np.dot(model['W2'], h)
    p = sigmoid(rho)
    return rho, p, h # return probability of taking action 2, and hidden state

The **policy_forward** function is essentially computing the activations of each neuron, given an input vector of pixels $\vec{X}$:

In [54]:
%%latex

\begin{align}
\left( 
\begin{array}{cccc}
w_{11} & w_{12} & \cdots & w_{1D}\\
w_{21} & w_{22} & \cdots & w_{2D}\\
\vdots & \vdots &   & \vdots\\
w_{H1} & w_{H2} & \cdots & w_{HD}\\    
\end{array} 
\right)

\times

\left(
\begin{array}{c}
X_{1} \\
X_{2} \\
\vdots \\
X_{D} \\    
\end{array}
\right)=

\left(
\begin{array}{c}
h_{1} \\
h_{2} \\
\vdots \\
h_{H} \\    
\end{array}
\right)
\end{align} 

<IPython.core.display.Latex object>

where $D$ and $H$ is the input size (number of pixels) and number of hidden neurons, respectively. The activation vector $\vec{h}$ contains the linear combination of the input pixels.

Next, we compute $\rho$ - the weighted sum of the activation vector $\vec{h}$ representing the aggregate signal from the neural net:

In [55]:
%%latex

\begin{align}
\left( 
\begin{array}{cccc}
\Omega_{1} & \Omega_{2} & \cdots & \Omega_{H}\\    
\end{array} 
\right)

\times

\left(
\begin{array}{c}
h_{1} \\
h_{2} \\
\vdots \\
h_{H} \\    
\end{array}
\right)= 

\rho

\end{align} 

<IPython.core.display.Latex object>

There are different ways in which to transform the aggregate signal $\rho$ into a probability $p$ for a certain action. Here, we use the sigmoid function. It has a few useful features: (i) $\rho$ = 0 means that the neural net  is indifferent to either action, hence the $p=0.5$, (ii) it is bounded in [0,1]

So essentially **policy_forward** is computing:

In [56]:
%%latex

\begin{align}
\text{sigmoid}
\left[
\left( 
\begin{array}{cccc}
\Omega_{1} & \Omega_{2} & \cdots & \Omega_{H}\\    
\end{array} 
\right)

\times
\left(
\underbrace{    
\left( 
\begin{array}{cccc}
w_{11} & w_{12} & \cdots & w_{1D}\\
w_{21} & w_{22} & \cdots & w_{2D}\\
\vdots & \vdots &   & \vdots\\
w_{H1} & w_{H2} & \cdots & w_{HD}\\    
\end{array} 
\right)
    
\times

\left(
\begin{array}{c}
X_{1} \\
X_{2} \\
\vdots \\
X_{D} \\    
\end{array}
\right)
}_{\vec{h}}
\right)
\right] = p
\end{align}

<IPython.core.display.Latex object>

### Backward pass

For the backward pass, we define an objective function at time $t$: 

$\Lambda(\Omega,W, t) = 0.5 \times R(t) \times(y-p)^2 + \dfrac{1}{2}\alpha \left(\sum_{i,j} w_{ij}^2\right)$, 

where $p$ is the probability to select an action (UP in this case) and $y$ is the actual action taken after sampling. In this sense $\Lambda$ represents the **uncertainty** or **inconsistency** of the neural network. 

$\Lambda \to 0$ implies that the neural network is confident in that it produces probabilities that are sufficiently close to the bounds of its support (i.e. 0 or 1). 

Note that the objective function is modulated by the reward at time $t$, $R(t)$. This has two consequences. First a big reward will serve as an incentive to increase confidence. In a sense the neural network will try to exploit the local landscape searching for a higher reward. Second, small reward will tend to render the network less sensitive to uncertainty. **This has the potential to cause convergence to a local suboptimum**.

Finally, we include L2 regularization in the last term with a penalty $\alpha$.

Our back-propagation algorithm will try to minimize $\Lambda$. Some math:

In [2]:
%%latex
\begin{align}

\dfrac{\partial \Lambda}{\partial \Omega_i} = \dfrac{1}{2}.R(t).
\dfrac{\partial (y-p)^2}{\partial \rho}.\dfrac{\partial \rho}{\partial \Omega_i}=
R(t).\dfrac{1}{2}\dfrac{\partial (y-\text{sigmoid}(\rho))^2}{\partial \rho}.
\dfrac{\partial (\Omega.\vec{h})}{\partial \Omega_i}\quad=\quad

-R(t).(y-p).\dfrac{d\text{sigmoid}(\rho)}{d\rho}.h_i
\end{align}



<IPython.core.display.Latex object>

and

In [12]:
%%latex

\begin{align}

\dfrac{\partial \Lambda}{\partial w_{ij}} = \dfrac{1}{2}.R(t).
\dfrac{\partial (y-p)^2}{\partial \rho}.\dfrac{\partial \rho}{\partial \vec{h}}.
\dfrac{\partial \vec{h}}{\partial w_{ij}}+
\dfrac{1}{2}\alpha\dfrac{\partial\left(\sum_{i,j} w_{ij}^2\right)}{\partial w_{ij}}

\quad=\quad

-R(t).(y-p).\dfrac{d\text{sigmoid}(\rho)}{d\rho}.\Omega_i.X_{j} + \alpha.w_{ij}

\end{align}

<IPython.core.display.Latex object>

In [9]:
def policy_backward(eph, epdlogp, epdrho):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp*epdrho).ravel()
    dh = np.outer(epdlogp*epdrho, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu.
    dW1 = np.dot(dh.T, epx) - alpha*model['W1']
    return {'W1':dW1, 'W2':dW2}

In [10]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)    
    running_add = 0
    for t in reversed(range(0, r.size)):
        if r[t] != 0: 
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

## Set-up 

In [45]:
# logging
logging_level = logging.DEBUG
flogging_level = logging.INFO

formatter = logging.Formatter(fmt=("%(asctime)s - %(levelname)s - %(module)s - %(message)s"))
shandler = logging.StreamHandler()
shandler.setFormatter(formatter)
shandler.setLevel(logging_level)

fhandler = logging.handlers.RotatingFileHandler('pong.log',encoding='utf-8',maxBytes=1e+8,backupCount=100)
fhandler.setFormatter(formatter)
fhandler.setLevel(flogging_level)

logger = logging.getLogger('PONG')
logger.setLevel(flogging_level)
logger.addHandler(shandler)
logger.addHandler(fhandler)

In [ ]:
### hyperparameters
config = configparser.ConfigParser()
config.read('config.properties')
H = int(config['NEURAL_NETWORK']['number_neurons']) # number of hidden layer neurons
batch_size = int(config['NEURAL_NETWORK']['batch_size']) # every how many episodes (games) to do a param update?
learning_rate = float(config['NEURAL_NETWORK']['learning_rate'])
gamma = float(config['NEURAL_NETWORK']['gamma']) # discount factor for reward
alpha = float(config['NEURAL_NETWORK']['alpha']) # L2 regularization
decay_rate = float(config['NEURAL_NETWORK']['decay_rate']) # decay factor for RMSProp leaky sum of grad^2
resume = config['NEURAL_NETWORK']['resume'].lower() == 'true' # resume from previous checkpoint?
render = config['NEURAL_NETWORK']['render'].lower() == 'true'
# number of pixels
D = int(config['NEURAL_NETWORK']['x']) # input dimensionality: 80x80 grid

# log the configs
logger.info('H:%d,D:%d,batch_size:%d,learning_rate:%f,gamma:%f,decay_rate:%f,resume:%s,render:%s',
           H,D,batch_size,learning_rate,gamma,decay_rate,resume,render)

if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    model['W2'] = np.random.randn(H) / np.sqrt(H)

'Xavier' initialization ensures that the variance of $W \times X$ equals the variance of $W$, i.e. the output of a neuron varies as much as the input $X$.
      
 Each element $w_{ij}$ of the matrix $W_1$ gives the weights of neuron $i$ in responding to input pixel $X_{j}$. I.e. the rows $w_i$ of $W_1$ correspond to the weights of neuron $i$ for the whole image.

In [ ]:
# update buffers that add up gradients over a batch
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } 
# rmsprop memory
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } 

prev_x = None # used in computing the difference frame
xs,hs,dlogps,drhos,drs = [],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
n_frames = 0 # number of frames in an episode

In [ ]:
#
env = gym.make("Pong-v0")
observation = env.reset()

# Have a look at the initial environment
plt.imshow(observation)
plt.show()

# Look at the available actions
env.get_action_meanings()
#
action_space = {'UP': 2, 'DOWN': 3}

'RIGHT' = 2 and 'LEFT' = 3 probably refer to 'UP' and 'DOWN' respectively

## Main loop

In [ ]:
keep_going = True
while keep_going:
    if render:
        env.render()

    # preprocess (crop) the observation, set input to network to be difference image
    cur_x = prepro(observation).astype(np.float).ravel()
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    # aprob is the probability of going UP
    rho, aprob, h = policy_forward(x)
    # UP = 2, DOWN = 3
    action = action_space['UP'] if np.random.uniform() < aprob else action_space['DOWN'] # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    drhos.append(dsigmoid(rho)) # the first derivative of the sigmoid activation function at rho
    y = 1 if action == action_space['UP'] else 0 # take an action

    """
     grad that encourages the action that was taken to be taken
     (see http://cs231n.github.io/neural-networks-2/#losses if confused)
    """
    dlogps.append(y - aprob)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)
    n_frames += 1

    if done: # an episode finished, i.e. score up to 21 for either player
        logger.info('Finished episode %d with %d frames.', episode_number, n_frames)
        
        episode_number += 1

        # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epdrho = np.vstack(drhos)
        epr = np.vstack(drs)
        xs,hs,dlogps,drhos,drs = [],[],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp, epdrho)
        for k in model:
            grad_buffer[k] += grad[k] # accumulate grad over batch

        # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            logger.info('Performing param update')
            for k,v in model.items():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer


        # boring book-keeping
        if episode_number % 100 == 0:
            logger.info("Writing model to file, episode %d",episode_number)
            pickle.dump(model, open('save_'+str(episode_number)+'.p', 'wb'))
        
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        logger.info('Resetting env. episode total/mean reward, up frequency and sum(W1): %f \t %f \t %f \t %f',
                    reward_sum, running_reward,model['frequency_up'],np.sum(model['W1']))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None

        if reward == 1 or reward == -1:
            logger.info('ep %d: game finished, reward: %f', episode_number, reward)
        else:
            logger.error('ep %d: game finished, but received invalid reward: %f !!!', episode_number, reward)
